In [1]:
# Alfredo Rojas
# 12/16/2020
# description: NDVI decadal extraction 
# filename: bf_ndvi_decadal.ipynb

import ee
import geemap
import os

os.chdir("/Users/alfredorojas/Dropbox/GitHub_2020/gee-practice/")
os.getcwd()

'/Users/alfredorojas/Dropbox/GitHub_2020/gee-practice'

In [2]:
Map = geemap.Map()
Map.setCenter(-2.142, 12.36)
Map

Map(center=[12.36, -2.142], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

GIMMS NDVI from AVHRR Sensors (3rd Generation) starts on 07-01-1981 and goes from 12-31-2013. Do the process for each round of DHS clusters. I'm not sure of a more elegant way to do this in GEE yet, so I'll just copy&paste 4 times :( The upside to this is that I can just re-run the script for seperate DHS rounds as opposed to doing it in batches. I will work on the workflow a little later.

Set up all of the data layers for extraction.

In [3]:
# 1981 to 1991
start_date_81 = '1981-07-01'
end_date_91 = '1991-12-31'

# 1992 to 2002
start_date_92 = '1992-01-01'
end_date_02 = '2002-12-31'

# 2003 to 2013
start_date_03 = '2003-01-01'
end_date_13 = '2013-12-31'

### Add Earth Engine dataset, select NDVI + filter
dataset1 = ee.ImageCollection('NASA/GIMMS/3GV0') \
        .filterDate(start_date_81, end_date_91) 

ndvi1 = dataset1.select('ndvi')

### add second dataset from 92 to 02
dataset2 = ee.ImageCollection('NASA/GIMMS/3GV0') \
        .filterDate(start_date_92, end_date_02) 

ndvi2 = dataset2.select('ndvi')

### Add third dataset from 03 to 13
dataset3 = ee.ImageCollection('NASA/GIMMS/3GV0') \
        .filterDate(start_date_03, end_date_13) 

ndvi3 = dataset3.select('ndvi')

# bring in shape files for 4 rounds of DHS in order to add 10 km buffers
cluster_shp_93 = "data/BF_1993_DHS_04032020_1547_134297/BFGE23FL/BFGE23FL.shp"
cluster_shp_98 = "data/BF_1998-99_DHS_04032020_1546_134297/BFGE32FL/BFGE32FL.shp"
cluster_shp_03 = "data/BF_2003_DHS_04032020_1546_134297/BFGE43FL/BFGE43FL.shp"
cluster_shp_10 = "data/BF_2010_DHS_04032020_1546_134297/BFGE61FL/BFGE61FL.shp"

clusters1 = geemap.shp_to_ee(cluster_shp_93)
clusters2 = geemap.shp_to_ee(cluster_shp_98)
clusters3 = geemap.shp_to_ee(cluster_shp_03)
clusters4 = geemap.shp_to_ee(cluster_shp_10)

# function to add 10km buffers around DHS clusters
def addBuffer(feature):
    return feature.buffer(10000)

# apply buffer 
buffers1 = clusters1.map(addBuffer)
buffers2 = clusters2.map(addBuffer)
buffers3 = clusters3.map(addBuffer)
buffers4 = clusters4.map(addBuffer)

Can visualize to make sure it works. . . Viz 1

In [4]:
#Set visualization parameters.
ndvi_vis = {
  'min': -1,
  'max': 1,
  'palette': ['000000', 'f5f5f5', '119701']}

# Add Earth Engine AVHRR ndvi to map
Map.addLayer(ndvi1, ndvi_vis, 'NDVI')

In [5]:
# 1993 buffers
Map.addLayer(clusters1, {}, 'DHSID')
Map.addLayer(buffers1, {}, 'buffer') # seems to be an error in Display above if I try to add more buffers below... why?

In [6]:
# 1998 buffers
Map.addLayer(clusters2, {}, 'DHSID')
Map.addLayer(buffers2, {}, 'buffer')

In [ ]:
# 2003 buffers
Map.addLayer(clusters3, {}, 'DHSID')
Map.addLayer(buffers3, {}, 'buffer')

In [ ]:
# 2010 buffers
Map.addLayer(clusters4, {}, 'DHSID')
Map.addLayer(buffers4, {}, 'buffer')

In [4]:
# In order to do Zonal Stats on NDVI time series, use collection.toBands():
ndvi_bands1 = ndvi1.toBands()
ndvi_bands2 = ndvi2.toBands()
ndvi_bands3 = ndvi3.toBands()

In [5]:
print(clusters4.size().getInfo())

573


In [8]:
print(ndvi_bands1.projection().nominalScale().getInfo())

9276.624232772796


In [9]:
# output directory for the below Zonal Stats. . . 
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

### Decadal for 1993 DHS

In [10]:
## MEANS 81 to 91 and 92 to 02
out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_81to91_mean.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers1, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_92to02_mean.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers1, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

## MAXIMUM 81 to 91 and 92 to 02 
out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_81to91_max.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers1, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_92to02_max.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers1, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

## MINIMUM 81 to 91 and 92 to 02
out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_81to91_min.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers1, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1993_ndvi_92to02_min.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers1, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_81to91_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_92to02_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_81to91_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_92to02_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_81to91_min.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1993_ndvi_92to02_min.csv


### Decadal for 1998 DHS

In [11]:
## MEANS 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_81to91_mean.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers2, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_92to02_mean.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers2, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_03to13_mean.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers2, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

## MAXIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_81to91_max.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers2, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_92to02_max.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers2, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_03to13_max.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers2, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

## MINIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_81to91_min.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers2, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_92to02_min.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers2, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS1998_ndvi_03to13_min.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers2, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_81to91_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_92to02_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_03to13_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_81to91_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_92to02_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_03to13_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS1998_ndvi_81to91_min.csv
Computing statistics ...
Generating UR

### Decadal for 2003 DHS

In [12]:
## MEANS 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_81to91_mean.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers3, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_92to02_mean.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers3, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_03to13_mean.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers3, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

## MAXIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_81to91_max.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers3, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_92to02_max.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers3, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_03to13_max.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers3, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

## MINIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_81to91_min.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers3, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_92to02_min.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers3, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2003_ndvi_03to13_min.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers3, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_81to91_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_92to02_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_03to13_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_81to91_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_92to02_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_03to13_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2003_ndvi_81to91_min.csv
Computing statistics ...
Generating UR

### Decadal for 2010 DHS

In [13]:
## MEANS 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_81to91_mean.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers4, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_92to02_mean.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers4, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_03to13_mean.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers4, out_ndvi_stats, statistics_type = 'MEAN', scale = 9276)

## MAXIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_81to91_max.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers4, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_92to02_max.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers4, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_03to13_max.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers4, out_ndvi_stats, statistics_type = 'MAXIMUM', scale = 9276)

## MINIMUM 81 to 91, 92 to 02, and 03 to 13
out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_81to91_min.csv')  
geemap.zonal_statistics(ndvi_bands1, buffers4, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_92to02_min.csv')  
geemap.zonal_statistics(ndvi_bands2, buffers4, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

out_ndvi_stats = os.path.join(out_dir, 'DHS2010_ndvi_03to13_min.csv')  
geemap.zonal_statistics(ndvi_bands3, buffers4, out_ndvi_stats, statistics_type = 'MINIMUM', scale = 9276)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_81to91_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_92to02_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_03to13_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_81to91_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_92to02_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_03to13_max.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/alfredorojas/Downloads/DHS2010_ndvi_81to91_min.csv
Computing statistics ...
Generating UR